Flashback and Familjeliv: The two Cultures
==========================================

In [ ]:
// %run /scalable-data-science/000_0-sds-3-x-projects/student-project-01_group-TheTwoCultures/00_download_data

In [ ]:
/scalable-data-science/000_0-sds-3-x-projects/student-project-01_group-TheTwoCultures/01_load_data

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/student-project-01/familjeliv")) 

  

[TABLE]

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/student-project-01/flashback"))

  

[TABLE]

  

>     import org.apache.spark.sql.functions.{col, concat_ws, udf, flatten, explode, collect_list, collect_set, lit}
>     import org.apache.spark.sql.types.{ArrayType, StructType, StructField, StringType, IntegerType}
>     import com.databricks.spark.xml._
>     import org.apache.spark.sql.functions._
>     read_xml: (file_name: String)org.apache.spark.sql.DataFrame
>     get_dataset: (file_name: String)org.apache.spark.sql.DataFrame

In [ ]:
val tmp_df = read_xml("dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad.xml")
tmp_df.printSchema()

  

>     root
>      |-- _id: string (nullable = false)
>      |-- _title: string (nullable = false)
>      |-- _url: string (nullable = false)
>      |-- thread: array (nullable = false)
>      |    |-- element: struct (containsNull = true)
>      |    |    |-- _id: string (nullable = false)
>      |    |    |-- _title: string (nullable = false)
>      |    |    |-- _url: string (nullable = false)
>      |    |    |-- text: struct (nullable = false)
>      |    |    |    |-- sentence: array (nullable = false)
>      |    |    |    |    |-- element: struct (containsNull = true)
>      |    |    |    |    |    |-- w: array (nullable = true)
>      |    |    |    |    |    |    |-- element: string (containsNull = true)
>
>     tmp_df: org.apache.spark.sql.DataFrame = [_id: string, _title: string ... 2 more fields]

In [ ]:
val file_name = "dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad.xml"
val df = get_dataset(file_name).cache()
val file_name2 = "dbfs:/datasets/student-project-01/flashback/flashback-sex.xml"
val df2 = get_dataset(file_name2).cache()

  

>     file_name: String = dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad.xml
>     df: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]
>     file_name2: String = dbfs:/datasets/student-project-01/flashback/flashback-sex.xml
>     df2: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]

In [ ]:
df.printSchema()

  

>     root
>      |-- thread_id: string (nullable = true)
>      |-- thread_title: string (nullable = true)
>      |-- w: string (nullable = false)
>      |-- forum_id: string (nullable = true)
>      |-- forum_title: string (nullable = true)
>      |-- platform: string (nullable = false)
>      |-- corpus_id: string (nullable = false)

In [ ]:
//display(df)

In [ ]:
//df.count

  

>     res42: Long = 7813

In [ ]:
import org.apache.spark.sql.functions.countDistinct
df.agg(countDistinct("thread_id"))

In [ ]:
import org.apache.spark.sql.types.{ LongType }
val df_test = df2.select("w", "thread_id").withColumn("thread_id",col("thread_id").cast(LongType))//.limit(100)//take(100).toDF()

  

>     import org.apache.spark.sql.types.LongType
>     df_test: org.apache.spark.sql.DataFrame = [w: string, thread_id: bigint]

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

val tokenizer = new RegexTokenizer()
  .setPattern("(?U)[\\W_]+") // break by white space character(s)  - try to remove emails and other patterns
  .setMinTokenLength(3) // Filter away tokens with length < 4
  .setInputCol("w") // name of the input column
  .setOutputCol("tokens") // name of the output column

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_8dfb36da5645, minTokenLength=3, gaps=true, pattern=(?U)[\W_]+, toLowercase=true

In [ ]:
df_test.printSchema()

  

>     root
>      |-- w: string (nullable = false)
>      |-- thread_id: long (nullable = true)

In [ ]:
val test_token = tokenizer.transform(df_test)

  

>     test_token: org.apache.spark.sql.DataFrame = [w: string, thread_id: bigint ... 1 more field]

In [ ]:
wget https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv



  

>     --2020-12-21 10:41:06--  https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1936 (1.9K) [text/plain]
>     Saving to: ‘stoppord.csv’
>
>          0K .                                                     100% 15.6M=0s
>
>     2020-12-21 10:41:06 (15.6 MB/s) - ‘stoppord.csv’ saved [1936/1936]

In [ ]:
dbutils.fs.cp("file:/databricks/driver/stoppord.csv", "dbfs:/datasets/student-project-01/stoppord.csv")
 
val stoppord = sc.textFile("dbfs:/datasets/student-project-01/stoppord.csv").collect()

  

>     stoppord: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)

In [ ]:
val stopwordList = Array("bara","lite","finns","vill","samt","inga","även","finns","ganska","också","igen","just","that","with","http","jpg").union(stoppord)

  

>     stopwordList: Array[String] = Array(bara, lite, finns, vill, samt, inga, även, finns, ganska, också, igen, just, that, with, http, jpg, word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)

In [ ]:
import org.apache.spark.ml.feature.StopWordsRemover


val remover = new StopWordsRemover()
  .setStopWords(stopwordList)
  .setInputCol("tokens")
  .setOutputCol("filtered")

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_fd37fe28b783, numStopWords=346, locale=en, caseSensitive=false

In [ ]:
val test_stop = remover.transform(test_token).select("thread_id","filtered")

  

>     test_stop: org.apache.spark.sql.DataFrame = [thread_id: bigint, filtered: array<string>]

In [ ]:
test_stop.printSchema()

  

>     root
>      |-- thread_id: long (nullable = true)
>      |-- filtered: array (nullable = true)
>      |    |-- element: string (containsNull = true)

In [ ]:
//test_stop.show(1, false)

In [ ]:
import org.apache.spark.ml.feature.CountVectorizer

val vectorizer = new CountVectorizer()
  .setInputCol("filtered")
  .setOutputCol("features")
  .setVocabSize(10000) // Size of dictonary
  .setMinDF(2) // the minimum number of different documents a term must appear in to be included in the vocabulary.
  .fit(test_stop)

  

>     import org.apache.spark.ml.feature.CountVectorizer
>     vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_70dec58904b4, vocabularySize=10000

In [ ]:
val countVectors = vectorizer.transform(test_stop)//.select("thread_id", "features")

  

>     countVectors: org.apache.spark.sql.DataFrame = [thread_id: bigint, filtered: array<string> ... 1 more field]

In [ ]:
//display(countVectors)//.show(1,false)

In [ ]:
//countVectors.count()

In [ ]:
//vectorizer.show(1, false)

In [ ]:
val countVectors = vectorizer.transform(test_stop).select("thread_id", "features")

  

>     countVectors: org.apache.spark.sql.DataFrame = [thread_id: bigint, features: vector]

In [ ]:
// Convert DF to RDD
import org.apache.spark.ml.linalg.Vector

val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }

val numTopics = 9

  

>     import org.apache.spark.ml.linalg.Vector
>     lda_countVector: org.apache.spark.sql.Dataset[(Long, org.apache.spark.ml.linalg.Vector)] = [_1: bigint, _2: vector]
>     numTopics: Int = 9

In [ ]:
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

// Set LDA params
val lda = new LDA()
.setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
.setK(numTopics)
.setMaxIterations(3)
.setDocConcentration(-1) // use default values
.setTopicConcentration(-1) // use default values

  

>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     lda: org.apache.spark.mllib.clustering.LDA = org.apache.spark.mllib.clustering.LDA@3294220d

In [ ]:
val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd

val ldaModel = lda.run(lda_countVector_mllib)

  

>     lda_countVector_mllib: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[15748] at rdd at command-4443336225071772:1
>     ldaModel: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@2af3bf

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)

  

>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(1245, 2867, 1, 1940, 0),Array(0.005828736389870819, 0.005177770692548582, 0.004281225684803414, 0.003802661243134556, 0.0037466618822440735)), (Array(2, 4, 0, 11, 119),Array(0.008472421218338572, 0.00835186048039426, 0.007028728954305847, 0.006925949553647179, 0.006778393528832462)), (Array(9, 1, 3, 81, 0),Array(0.007136098623257581, 0.006219005095591811, 0.00607101967985819, 0.005716841064824449, 0.00547781968634569)), (Array(90, 1480, 267, 379, 3),Array(0.012209939662717557, 0.010448534134872305, 0.00871415633044254, 0.004717316738793793, 0.0036140424647016466)), (Array(1, 4, 42, 2, 7),Array(0.008136396956028923, 0.007598480888602129, 0.0066038668672708575, 0.005629407585564553, 0.004621223223798453)), (Array(0, 3, 2, 1, 4),Array(0.008977479783453209, 0.00818731006415827, 0.007351349890882013, 0.007278806387254452, 0.005765657444255816)), (Array(11, 16, 4, 0, 7),Array(0.010225491273158971, 0.009673480686118726, 0.007140266582380782, 0.006741668845739555, 0.006166286203067536)), (Array(5, 2, 4, 3, 6),Array(0.007127353527224035, 0.004957423825932039, 0.004956635485595906, 0.004879988547583389, 0.004864308362389227)), (Array(1, 0, 3, 2, 4),Array(0.008605481820265399, 0.007793354439640682, 0.007702749345394846, 0.007368449734767341, 0.007331716366632319)))

In [ ]:
val vocabList = vectorizer.vocabulary

  

>     vocabList: Array[String] = Array(mycket, kanske, tjejer, bra, tycker, tjej, aldrig, många, måste, fan, rätt, kvinnor, killar, samma, gillar, väldigt, män, riktigt, känner, kille, tjejen, alltid, verkar, bättre, fel, faktiskt, problem, lika, folk, känns, fråga, ens, typ, jävla, knulla, olika, kvinna, precis, brukar, behöver, svårt, håller, barn, borde, kuk, runt, stor, låter, saker, vissa, porr, tråden, haha, sett, ofta, mest, enkelt, människor, klart, tänka, tänker, annars, egentligen, istället, direkt, killen, hålla, förstår, länge, mindre, alls, osv, sak, dessutom, skönt, liten, börjar, säkert, vilja, själva, handlar, bilder, tänder, stora, kuken, hitta, ville, hem, bort, såg, the, kul, nästan, däremot, iaf, större, känna, varandra, kåt, bästa, tag, lätt, hos, längre, tråd, förhållande, enda, sitter, ihop, egen, svar, tjejerna, sånt, inlägg, dra, sängen, massa, tillsammans, kolla, bröst, pengar, veta, fortfarande, tidigare, liv, funkar, fitta, flickvän, person, liknande, enligt, hoppas, vanligt, tyvärr, börja, ollon, speciellt, par, älskar, liksom, lär, svenska, orgasm, händer, kropp, ont, absolut, frågan, beror, tips, emot, jävligt, snygg, sidan, oftast, långt, först, prata, undrar, skit, vore, ifrån, tillbaka, partner, sjukt, bör, slut, självklart, började, sexuella, testa, livet, sluta, försöka, tänkte, spelar, försöker, dåligt, visa, hört, filmer, okej, runka, köpa, trots, knullar, brudar, skrev, gånger, roll, film, intressant, små, äldre, etc, tid, problemet, köra, gången, hårt, konstigt, kvinnan, hemma, brud, suga, bild, sida, hand, gott, fittan, tyckte, tänk, hittar, visar, anser, sätta, tycka, hel, ungefär, rent, sexuellt, bäst, snarare, erfarenhet, bajs, kondom, skillnad, pratar, låta, egna, berätta, början, slicka, särskilt, dvs, snygga, knappast, stort, höra, trodde, igång, illa, använda, drar, munnen, personer, kompis, övrigt, ord, oavsett, förstå, mannen, kroppen, intresserad, enbart, filmen, hår, extra, satt, sån, endast, sexet, fungerar, hej, givetvis, pga, fattar, and, analsex, träffat, normalt, vänner, gammal, använder, oskuld, läsa, skriva, titta, möjligt, röven, träffa, krogen, medan, exempel, personen, tanke, anledning, ålder, exakt, läst, tydligen, låt, bryr, yngre, känslor, svara, lägger, snabbt, hora, vanliga, söker, förmodligen, fina, stämmer, unga, personligen, fort, minst, efteråt, antar, prova, lilla, leva, betala, människa, låg, frågar, nått, hör, sant, polare, bor, hänt, ansiktet, världen, otroligt, säker, ifall, ingenting, betyder, tanken, huvudet, antagligen, mkt, väljer, ena, synd, föredrar, äckligt, tur, jobbigt, flashback, såklart, gamla, dålig, helvete, jobb, full, lättare, tagit, lycka, stå, saken, sätter, for, suger, trevligt, kort, riktig, kände, pojkvän, prostitution, träffar, form, välja, nära, säg, sexuell, tänkt, samlag, ung, minns, tex, grej, försök, lever, extremt, ögon, kontakt, you, glad, köper, skaffa, bög, lära, mår, igenom, bilden, kön, runkar, viss, bilderna, nätet, antingen, prostituerade, penis, skull, relation, namn, hehe, svensk, hittade, roligt, vanlig, sexualitet, trevlig, kollar, kärlek, förklara, visste, lång, ihåg, viktigt, vågar, såna, tittar, senare, naturligtvis, behöva, homosexuella, kåta, vänta, åka, jobbar, hjälpa, kalla, fint, alldeles, njutning, rädd, utseende, håll, sperma, sexigt, forum, bögar, förutom, iallafall, totalt, luktar, fortsätta, horor, sällan, internet, värt, ful, vän, ensam, klipp, seriöst, naturligt, fula, snacka, upphetsande, mesta, kär, njuta, orkar, sälja, stånd, tillräckligt, aning, kukar, testat, ordentligt, ställer, räcker, djur, knappt, medans, handen, erfarenheter, nere, tackar, fullt, reda, skrivit, nytt, ställa, sidor, sådär, frågade, sugen, bruden, kring, lyckas, någonsin, kompisar, mod, känsla, ämnet, jobbet, läste, tvärtom, läser, innebär, nöjd, klarar, mej, största, normal, värsta, känslan, sättet, princip, tiden, resten, vägen, via, oralsex, betydligt, riktiga, dum, snackar, hahaha, utav, antal, tråkigt, behov, knullat, råd, risken, benen, intresse, plötsligt, frågor, förut, passar, iofs, fru, tankar, hittat, drog, mamma, situation, tala, ställe, talar, hände, njuter, gratis, slags, väll, chans, gilla, vis, samhället, kläder, annorlunda, plats, lust, förstås, eget, ovanligt, säljer, litet, kändes, god, delar, träffade, föräldrar, brösten, självförtroende, killarna, jobba, spela, hända, definitivt, åker, höll, beroende, börjat, kunder, verkligheten, hmm, perfekt, trådar, naken, våldtäkt, avsugning, slipper, betalar, åsikter, fantasier, skön, skiter, söt, märker, avtändande, leta, åring, grund, rum, stället, ren, ansikte, kvinnorna, fallet, spruta, lugnt, finner, läs, gav, grejen, flickor, syn, ställen, väg, generellt, ärligt, ons, långa, sexliv, hänger, situationen, vuxna, hård, förhållanden, nyfiken, beteende, krav, kvinnliga, val, piller, porrfilm, äkta, forumet, svaret, överhuvudtaget, anledningen, någonstans, länk, dela, äta, röv, grymt, knull, kallar, tja, mun, manliga, ragga, otrogen, oxå, länder, spännande, omöjligt, söka, mvh, risk, sker, vacker, svårare, sexig, rör, stan, stoppa, åsikt, iväg, åldern, menade, trött, umgås, säng, försökte, diskutera, tända, rejält, snälla, mat, lägg, förresten, slå, fingrar, hjälper, slutar, nice, svarar, rumpa, trycka, underbart, provat, ses, ben, ber, pass, uppenbarligen, topic, massor, stod, skicka, såhär, männen, fet, saknar, per, leka, dagens, funka, glidmedel, smal, knullade, bakifrån, kännas, skillnaden, därmed, dags, betalt, sambo, allmänt, faktum, lyckats, kallas, högre, nja, försökt, skönare, lagom, hög, vafan, dildo, gillade, hatar, plus, fixa, alternativ, lagen, talat, omkring, slickar, vackra, känt, länkar, krävs, ligg, homosexualitet, land, kostar, förspel, sjuk, osäker, tvungen, rumpan, upphetsad, räknas, koll, slår, klitoris, härligt, smak, ovan, magen, fortsätter, flicka, fantasi, hål, bry, acceptera, klara, länken, håret, främst, dels, chansen, vita, sova, ögonen, massage, skiten, allra, damer, önskar, snyggt, inser, normala, ngn, dåliga, visade, sexköp, minsta, orgasmer, 100, alkohol, utlösning, respekt, vatten, läggning, fundera, leder, stil, porren, skolan, ordet, tydligt, sjuka, sämre, huvud, oskulden, exempelvis, diverse, lät, pinsamt, förhållandet, lagt, polisen, vare, desto, fysiskt, hiv, hållet, argument, förslag, svart, fötter, borta, angående, öppna, bdsm, tjänster, byta, analt, träffas, viagra, nämligen, velat, gränsen, råkar, sorts, pengarna, ollonet, satte, relativt, uppskattar, bad, kräver, påstå, storlek, olagligt, världens, oerhört, het, klippet, trekant, pungen, ladda, slutade, senaste, äter, smakar, ansvar, dej, ngt, fittor, lämna, sexiga, satsen, relationer, seriös, pappa, snubbe, rolig, märkt, träna, väntar, rätta, raka, skada, akten, bla, körde, närheten, funderar, uppe, rummet, slappna, fyfan, upplever, släppa, halsen, allvar, far, prostituerad, förstått, snäll, övergrepp, lagligt, behövs, inse, barnet, berättar, tvåan, garanterat, partners, idé, köpt, dominant, räkna, faller, letar, besök, farligt, köp, rygg, kollade, hårdare, könet, förr, erkänna, smeka, svarta, ändra, inlägget, dörren, påstår, upplevelse, info, lider, sms, nakna, högt, ringa, finger, visserligen, mens, stark, starkt, ryggen, vuxen, majoriteten, beskriver, mage, därifrån, anal, familj, personlighet, idiot, öppet, hamnar, nivå, förbi, värld, shit, typen, kroppar, filmerna, äcklig, runk, escort, ärlig, datorn, överallt, fest, fungera, mening, äckliga, samhälle, trosor, röra, pratat, skäl, fatta, onanera, feta, öppen, duschen, trycker, pratade, one, tom, berättade, likadant, sprutar, troligen, gravid, undvika, hemsida, onani, smala, nuvarande, quote, undra, dumt, sök, lärt, våt, längden, brudarna, ledsen, porrfilmer, fullständigt, slippa, muskler, årig, verkade, spoiler, bestämma, död, smaken, usch, dricka)

In [ ]:
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}

  

>     topics: Array[Array[(String, Double)]] = Array(Array((com,0.005828736389870819), (font,0.005177770692548582), (kanske,0.004281225684803414), (size,0.003802661243134556), (mycket,0.0037466618822440735)), Array((tjejer,0.008472421218338572), (tycker,0.00835186048039426), (mycket,0.007028728954305847), (kvinnor,0.006925949553647179), (bröst,0.006778393528832462)), Array((fan,0.007136098623257581), (kanske,0.006219005095591811), (bra,0.00607101967985819), (bilder,0.005716841064824449), (mycket,0.00547781968634569)), Array((the,0.012209939662717557), (datanördar,0.010448534134872305), (and,0.00871415633044254), (you,0.004717316738793793), (bra,0.0036140424647016466)), Array((kanske,0.008136396956028923), (tycker,0.007598480888602129), (barn,0.0066038668672708575), (tjejer,0.005629407585564553), (många,0.004621223223798453)), Array((mycket,0.008977479783453209), (bra,0.00818731006415827), (tjejer,0.007351349890882013), (kanske,0.007278806387254452), (tycker,0.005765657444255816)), Array((kvinnor,0.010225491273158971), (män,0.009673480686118726), (tycker,0.007140266582380782), (mycket,0.006741668845739555), (många,0.006166286203067536)), Array((tjej,0.007127353527224035), (tjejer,0.004957423825932039), (tycker,0.004956635485595906), (bra,0.004879988547583389), (aldrig,0.004864308362389227)), Array((kanske,0.008605481820265399), (mycket,0.007793354439640682), (bra,0.007702749345394846), (tjejer,0.007368449734767341), (tycker,0.007331716366632319)))

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 10)
val vocabList = vectorizer.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     9 topics:
>     TOPIC 0
>     com	0.005828736389870819
>     font	0.005177770692548582
>     kanske	0.004281225684803414
>     size	0.003802661243134556
>     mycket	0.0037466618822440735
>     tycker	0.0035788236930912945
>     tjej	0.003572148658558219
>     wmv	0.003492052477056438
>     fan	0.003387032075470724
>     killar	0.0030759438164969536
>     ==========
>     TOPIC 1
>     tjejer	0.008472421218338572
>     tycker	0.00835186048039426
>     mycket	0.007028728954305847
>     kvinnor	0.006925949553647179
>     bröst	0.006778393528832462
>     kanske	0.006562072420550684
>     bra	0.005735098526572583
>     tjej	0.00509426741049249
>     många	0.0048473374351092275
>     stora	0.00476766751743259
>     ==========
>     TOPIC 2
>     fan	0.007136098623257581
>     kanske	0.006219005095591811
>     bra	0.00607101967985819
>     bilder	0.005716841064824449
>     mycket	0.00547781968634569
>     tjej	0.005384256034422819
>     tycker	0.005334349560442397
>     tjejer	0.004803912617599809
>     riktigt	0.004136561405063082
>     måste	0.004049538541763246
>     ==========
>     TOPIC 3
>     the	0.012209939662717557
>     datanördar	0.010448534134872305
>     and	0.00871415633044254
>     you	0.004717316738793793
>     bra	0.0036140424647016466
>     kanske	0.002794974212182175
>     for	0.00257613484350404
>     mycket	0.0025543316673021775
>     tjejer	0.0025314211222808034
>     fan	0.0021470545469380897
>     ==========
>     TOPIC 4
>     kanske	0.008136396956028923
>     tycker	0.007598480888602129
>     barn	0.0066038668672708575
>     tjejer	0.005629407585564553
>     många	0.004621223223798453
>     aldrig	0.004402198423276141
>     bra	0.004294088390266522
>     mycket	0.004118530511479333
>     fan	0.004054347059086492
>     rätt	0.003984365147630218
>     ==========
>     TOPIC 5
>     mycket	0.008977479783453209
>     bra	0.00818731006415827
>     tjejer	0.007351349890882013
>     kanske	0.007278806387254452
>     tycker	0.005765657444255816
>     tjej	0.005283260728111136
>     aldrig	0.004850961525403287
>     måste	0.004295311331401175
>     många	0.004098909420309231
>     fan	0.0037519527610768363
>     ==========
>     TOPIC 6
>     kvinnor	0.010225491273158971
>     män	0.009673480686118726
>     tycker	0.007140266582380782
>     mycket	0.006741668845739555
>     många	0.006166286203067536
>     tjejer	0.0055899906619138835
>     kanske	0.005496960145787021
>     bra	0.00450516316658287
>     rätt	0.003975370248418236
>     samma	0.003745046376784121
>     ==========
>     TOPIC 7
>     tjej	0.007127353527224035
>     tjejer	0.004957423825932039
>     tycker	0.004956635485595906
>     bra	0.004879988547583389
>     aldrig	0.004864308362389227
>     kanske	0.004588326800005052
>     fan	0.0045304366858314
>     mycket	0.004254791775847753
>     kille	0.003227585234792242
>     många	0.003104815800968856
>     ==========
>     TOPIC 8
>     kanske	0.008605481820265399
>     mycket	0.007793354439640682
>     bra	0.007702749345394846
>     tjejer	0.007368449734767341
>     tycker	0.007331716366632319
>     tjej	0.007143594626213114
>     aldrig	0.0057960981929391165
>     måste	0.004375258913740614
>     killar	0.004267007182925261
>     känner	0.004083723539468458
>     ==========
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(1245, 2867, 1, 1940, 0, 4, 5, 2415, 9, 12),Array(0.005828736389870819, 0.005177770692548582, 0.004281225684803414, 0.003802661243134556, 0.0037466618822440735, 0.0035788236930912945, 0.003572148658558219, 0.003492052477056438, 0.003387032075470724, 0.0030759438164969536)), (Array(2, 4, 0, 11, 119, 1, 3, 5, 7, 83),Array(0.008472421218338572, 0.00835186048039426, 0.007028728954305847, 0.006925949553647179, 0.006778393528832462, 0.006562072420550684, 0.005735098526572583, 0.00509426741049249, 0.0048473374351092275, 0.00476766751743259)), (Array(9, 1, 3, 81, 0, 5, 4, 2, 17, 8),Array(0.007136098623257581, 0.006219005095591811, 0.00607101967985819, 0.005716841064824449, 0.00547781968634569, 0.005384256034422819, 0.005334349560442397, 0.004803912617599809, 0.004136561405063082, 0.004049538541763246)), (Array(90, 1480, 267, 379, 3, 1, 354, 0, 2, 9),Array(0.012209939662717557, 0.010448534134872305, 0.00871415633044254, 0.004717316738793793, 0.0036140424647016466, 0.002794974212182175, 0.00257613484350404, 0.0025543316673021775, 0.0025314211222808034, 0.0021470545469380897)), (Array(1, 4, 42, 2, 7, 6, 3, 0, 9, 10),Array(0.008136396956028923, 0.007598480888602129, 0.0066038668672708575, 0.005629407585564553, 0.004621223223798453, 0.004402198423276141, 0.004294088390266522, 0.004118530511479333, 0.004054347059086492, 0.003984365147630218)), (Array(0, 3, 2, 1, 4, 5, 6, 8, 7, 9),Array(0.008977479783453209, 0.00818731006415827, 0.007351349890882013, 0.007278806387254452, 0.005765657444255816, 0.005283260728111136, 0.004850961525403287, 0.004295311331401175, 0.004098909420309231, 0.0037519527610768363)), (Array(11, 16, 4, 0, 7, 2, 1, 3, 10, 13),Array(0.010225491273158971, 0.009673480686118726, 0.007140266582380782, 0.006741668845739555, 0.006166286203067536, 0.0055899906619138835, 0.005496960145787021, 0.00450516316658287, 0.003975370248418236, 0.003745046376784121)), (Array(5, 2, 4, 3, 6, 1, 9, 0, 19, 7),Array(0.007127353527224035, 0.004957423825932039, 0.004956635485595906, 0.004879988547583389, 0.004864308362389227, 0.004588326800005052, 0.0045304366858314, 0.004254791775847753, 0.003227585234792242, 0.003104815800968856)), (Array(1, 0, 3, 2, 4, 5, 6, 8, 12, 18),Array(0.008605481820265399, 0.007793354439640682, 0.007702749345394846, 0.007368449734767341, 0.007331716366632319, 0.007143594626213114, 0.0057960981929391165, 0.004375258913740614, 0.004267007182925261, 0.004083723539468458)))
>     vocabList: Array[String] = Array(mycket, kanske, tjejer, bra, tycker, tjej, aldrig, många, måste, fan, rätt, kvinnor, killar, samma, gillar, väldigt, män, riktigt, känner, kille, tjejen, alltid, verkar, bättre, fel, faktiskt, problem, lika, folk, känns, fråga, ens, typ, jävla, knulla, olika, kvinna, precis, brukar, behöver, svårt, håller, barn, borde, kuk, runt, stor, låter, saker, vissa, porr, tråden, haha, sett, ofta, mest, enkelt, människor, klart, tänka, tänker, annars, egentligen, istället, direkt, killen, hålla, förstår, länge, mindre, alls, osv, sak, dessutom, skönt, liten, börjar, säkert, vilja, själva, handlar, bilder, tänder, stora, kuken, hitta, ville, hem, bort, såg, the, kul, nästan, däremot, iaf, större, känna, varandra, kåt, bästa, tag, lätt, hos, längre, tråd, förhållande, enda, sitter, ihop, egen, svar, tjejerna, sånt, inlägg, dra, sängen, massa, tillsammans, kolla, bröst, pengar, veta, fortfarande, tidigare, liv, funkar, fitta, flickvän, person, liknande, enligt, hoppas, vanligt, tyvärr, börja, ollon, speciellt, par, älskar, liksom, lär, svenska, orgasm, händer, kropp, ont, absolut, frågan, beror, tips, emot, jävligt, snygg, sidan, oftast, långt, först, prata, undrar, skit, vore, ifrån, tillbaka, partner, sjukt, bör, slut, självklart, började, sexuella, testa, livet, sluta, försöka, tänkte, spelar, försöker, dåligt, visa, hört, filmer, okej, runka, köpa, trots, knullar, brudar, skrev, gånger, roll, film, intressant, små, äldre, etc, tid, problemet, köra, gången, hårt, konstigt, kvinnan, hemma, brud, suga, bild, sida, hand, gott, fittan, tyckte, tänk, hittar, visar, anser, sätta, tycka, hel, ungefär, rent, sexuellt, bäst, snarare, erfarenhet, bajs, kondom, skillnad, pratar, låta, egna, berätta, början, slicka, särskilt, dvs, snygga, knappast, stort, höra, trodde, igång, illa, använda, drar, munnen, personer, kompis, övrigt, ord, oavsett, förstå, mannen, kroppen, intresserad, enbart, filmen, hår, extra, satt, sån, endast, sexet, fungerar, hej, givetvis, pga, fattar, and, analsex, träffat, normalt, vänner, gammal, använder, oskuld, läsa, skriva, titta, möjligt, röven, träffa, krogen, medan, exempel, personen, tanke, anledning, ålder, exakt, läst, tydligen, låt, bryr, yngre, känslor, svara, lägger, snabbt, hora, vanliga, söker, förmodligen, fina, stämmer, unga, personligen, fort, minst, efteråt, antar, prova, lilla, leva, betala, människa, låg, frågar, nått, hör, sant, polare, bor, hänt, ansiktet, världen, otroligt, säker, ifall, ingenting, betyder, tanken, huvudet, antagligen, mkt, väljer, ena, synd, föredrar, äckligt, tur, jobbigt, flashback, såklart, gamla, dålig, helvete, jobb, full, lättare, tagit, lycka, stå, saken, sätter, for, suger, trevligt, kort, riktig, kände, pojkvän, prostitution, träffar, form, välja, nära, säg, sexuell, tänkt, samlag, ung, minns, tex, grej, försök, lever, extremt, ögon, kontakt, you, glad, köper, skaffa, bög, lära, mår, igenom, bilden, kön, runkar, viss, bilderna, nätet, antingen, prostituerade, penis, skull, relation, namn, hehe, svensk, hittade, roligt, vanlig, sexualitet, trevlig, kollar, kärlek, förklara, visste, lång, ihåg, viktigt, vågar, såna, tittar, senare, naturligtvis, behöva, homosexuella, kåta, vänta, åka, jobbar, hjälpa, kalla, fint, alldeles, njutning, rädd, utseende, håll, sperma, sexigt, forum, bögar, förutom, iallafall, totalt, luktar, fortsätta, horor, sällan, internet, värt, ful, vän, ensam, klipp, seriöst, naturligt, fula, snacka, upphetsande, mesta, kär, njuta, orkar, sälja, stånd, tillräckligt, aning, kukar, testat, ordentligt, ställer, räcker, djur, knappt, medans, handen, erfarenheter, nere, tackar, fullt, reda, skrivit, nytt, ställa, sidor, sådär, frågade, sugen, bruden, kring, lyckas, någonsin, kompisar, mod, känsla, ämnet, jobbet, läste, tvärtom, läser, innebär, nöjd, klarar, mej, största, normal, värsta, känslan, sättet, princip, tiden, resten, vägen, via, oralsex, betydligt, riktiga, dum, snackar, hahaha, utav, antal, tråkigt, behov, knullat, råd, risken, benen, intresse, plötsligt, frågor, förut, passar, iofs, fru, tankar, hittat, drog, mamma, situation, tala, ställe, talar, hände, njuter, gratis, slags, väll, chans, gilla, vis, samhället, kläder, annorlunda, plats, lust, förstås, eget, ovanligt, säljer, litet, kändes, god, delar, träffade, föräldrar, brösten, självförtroende, killarna, jobba, spela, hända, definitivt, åker, höll, beroende, börjat, kunder, verkligheten, hmm, perfekt, trådar, naken, våldtäkt, avsugning, slipper, betalar, åsikter, fantasier, skön, skiter, söt, märker, avtändande, leta, åring, grund, rum, stället, ren, ansikte, kvinnorna, fallet, spruta, lugnt, finner, läs, gav, grejen, flickor, syn, ställen, väg, generellt, ärligt, ons, långa, sexliv, hänger, situationen, vuxna, hård, förhållanden, nyfiken, beteende, krav, kvinnliga, val, piller, porrfilm, äkta, forumet, svaret, överhuvudtaget, anledningen, någonstans, länk, dela, äta, röv, grymt, knull, kallar, tja, mun, manliga, ragga, otrogen, oxå, länder, spännande, omöjligt, söka, mvh, risk, sker, vacker, svårare, sexig, rör, stan, stoppa, åsikt, iväg, åldern, menade, trött, umgås, säng, försökte, diskutera, tända, rejält, snälla, mat, lägg, förresten, slå, fingrar, hjälper, slutar, nice, svarar, rumpa, trycka, underbart, provat, ses, ben, ber, pass, uppenbarligen, topic, massor, stod, skicka, såhär, männen, fet, saknar, per, leka, dagens, funka, glidmedel, smal, knullade, bakifrån, kännas, skillnaden, därmed, dags, betalt, sambo, allmänt, faktum, lyckats, kallas, högre, nja, försökt, skönare, lagom, hög, vafan, dildo, gillade, hatar, plus, fixa, alternativ, lagen, talat, omkring, slickar, vackra, känt, länkar, krävs, ligg, homosexualitet, land, kostar, förspel, sjuk, osäker, tvungen, rumpan, upphetsad, räknas, koll, slår, klitoris, härligt, smak, ovan, magen, fortsätter, flicka, fantasi, hål, bry, acceptera, klara, länken, håret, främst, dels, chansen, vita, sova, ögonen, massage, skiten, allra, damer, önskar, snyggt, inser, normala, ngn, dåliga, visade, sexköp, minsta, orgasmer, 100, alkohol, utlösning, respekt, vatten, läggning, fundera, leder, stil, porren, skolan, ordet, tydligt, sjuka, sämre, huvud, oskulden, exempelvis, diverse, lät, pinsamt, förhållandet, lagt, polisen, vare, desto, fysiskt, hiv, hållet, argument, förslag, svart, fötter, borta, angående, öppna, bdsm, tjänster, byta, analt, träffas, viagra, nämligen, velat, gränsen, råkar, sorts, pengarna, ollonet, satte, relativt, uppskattar, bad, kräver, påstå, storlek, olagligt, världens, oerhört, het, klippet, trekant, pungen, ladda, slutade, senaste, äter, smakar, ansvar, dej, ngt, fittor, lämna, sexiga, satsen, relationer, seriös, pappa, snubbe, rolig, märkt, träna, väntar, rätta, raka, skada, akten, bla, körde, närheten, funderar, uppe, rummet, slappna, fyfan, upplever, släppa, halsen, allvar, far, prostituerad, förstått, snäll, övergrepp, lagligt, behövs, inse, barnet, berättar, tvåan, garanterat, partners, idé, köpt, dominant, räkna, faller, letar, besök, farligt, köp, rygg, kollade, hårdare, könet, förr, erkänna, smeka, svarta, ändra, inlägget, dörren, påstår, upplevelse, info, lider, sms, nakna, högt, ringa, finger, visserligen, mens, stark, starkt, ryggen, vuxen, majoriteten, beskriver, mage, därifrån, anal, familj, personlighet, idiot, öppet, hamnar, nivå, förbi, värld, shit, typen, kroppar, filmerna, äcklig, runk, escort, ärlig, datorn, överallt, fest, fungera, mening, äckliga, samhälle, trosor, röra, pratat, skäl, fatta, onanera, feta, öppen, duschen, trycker, pratade, one, tom, berättade, likadant, sprutar, troligen, gravid, undvika, hemsida, onani, smala, nuvarande, quote, undra, dumt, sök, lärt, våt, längden, brudarna, ledsen, porrfilmer, fullständigt, slippa, muskler, årig, verkade, spoiler, bestämma, död, smaken, usch, dricka)
>     topics: Array[Array[(String, Double)]] = Array(Array((com,0.005828736389870819), (font,0.005177770692548582), (kanske,0.004281225684803414), (size,0.003802661243134556), (mycket,0.0037466618822440735), (tycker,0.0035788236930912945), (tjej,0.003572148658558219), (wmv,0.003492052477056438), (fan,0.003387032075470724), (killar,0.0030759438164969536)), Array((tjejer,0.008472421218338572), (tycker,0.00835186048039426), (mycket,0.007028728954305847), (kvinnor,0.006925949553647179), (bröst,0.006778393528832462), (kanske,0.006562072420550684), (bra,0.005735098526572583), (tjej,0.00509426741049249), (många,0.0048473374351092275), (stora,0.00476766751743259)), Array((fan,0.007136098623257581), (kanske,0.006219005095591811), (bra,0.00607101967985819), (bilder,0.005716841064824449), (mycket,0.00547781968634569), (tjej,0.005384256034422819), (tycker,0.005334349560442397), (tjejer,0.004803912617599809), (riktigt,0.004136561405063082), (måste,0.004049538541763246)), Array((the,0.012209939662717557), (datanördar,0.010448534134872305), (and,0.00871415633044254), (you,0.004717316738793793), (bra,0.0036140424647016466), (kanske,0.002794974212182175), (for,0.00257613484350404), (mycket,0.0025543316673021775), (tjejer,0.0025314211222808034), (fan,0.0021470545469380897)), Array((kanske,0.008136396956028923), (tycker,0.007598480888602129), (barn,0.0066038668672708575), (tjejer,0.005629407585564553), (många,0.004621223223798453), (aldrig,0.004402198423276141), (bra,0.004294088390266522), (mycket,0.004118530511479333), (fan,0.004054347059086492), (rätt,0.003984365147630218)), Array((mycket,0.008977479783453209), (bra,0.00818731006415827), (tjejer,0.007351349890882013), (kanske,0.007278806387254452), (tycker,0.005765657444255816), (tjej,0.005283260728111136), (aldrig,0.004850961525403287), (måste,0.004295311331401175), (många,0.004098909420309231), (fan,0.0037519527610768363)), Array((kvinnor,0.010225491273158971), (män,0.009673480686118726), (tycker,0.007140266582380782), (mycket,0.006741668845739555), (många,0.006166286203067536), (tjejer,0.0055899906619138835), (kanske,0.005496960145787021), (bra,0.00450516316658287), (rätt,0.003975370248418236), (samma,0.003745046376784121)), Array((tjej,0.007127353527224035), (tjejer,0.004957423825932039), (tycker,0.004956635485595906), (bra,0.004879988547583389), (aldrig,0.004864308362389227), (kanske,0.004588326800005052), (fan,0.0045304366858314), (mycket,0.004254791775847753), (kille,0.003227585234792242), (många,0.003104815800968856)), Array((kanske,0.008605481820265399), (mycket,0.007793354439640682), (bra,0.007702749345394846), (tjejer,0.007368449734767341), (tycker,0.007331716366632319), (tjej,0.007143594626213114), (aldrig,0.0057960981929391165), (måste,0.004375258913740614), (killar,0.004267007182925261), (känner,0.004083723539468458)))

In [ ]:
"det","att",

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

  

>     termArray: Array[(Array[(String, Double)], Int)] = Array((Array((com,0.005828736389870819), (font,0.005177770692548582), (kanske,0.004281225684803414), (size,0.003802661243134556), (mycket,0.0037466618822440735), (tycker,0.0035788236930912945), (tjej,0.003572148658558219), (wmv,0.003492052477056438), (fan,0.003387032075470724), (killar,0.0030759438164969536)),0), (Array((tjejer,0.008472421218338572), (tycker,0.00835186048039426), (mycket,0.007028728954305847), (kvinnor,0.006925949553647179), (bröst,0.006778393528832462), (kanske,0.006562072420550684), (bra,0.005735098526572583), (tjej,0.00509426741049249), (många,0.0048473374351092275), (stora,0.00476766751743259)),1), (Array((fan,0.007136098623257581), (kanske,0.006219005095591811), (bra,0.00607101967985819), (bilder,0.005716841064824449), (mycket,0.00547781968634569), (tjej,0.005384256034422819), (tycker,0.005334349560442397), (tjejer,0.004803912617599809), (riktigt,0.004136561405063082), (måste,0.004049538541763246)),2), (Array((the,0.012209939662717557), (datanördar,0.010448534134872305), (and,0.00871415633044254), (you,0.004717316738793793), (bra,0.0036140424647016466), (kanske,0.002794974212182175), (for,0.00257613484350404), (mycket,0.0025543316673021775), (tjejer,0.0025314211222808034), (fan,0.0021470545469380897)),3), (Array((kanske,0.008136396956028923), (tycker,0.007598480888602129), (barn,0.0066038668672708575), (tjejer,0.005629407585564553), (många,0.004621223223798453), (aldrig,0.004402198423276141), (bra,0.004294088390266522), (mycket,0.004118530511479333), (fan,0.004054347059086492), (rätt,0.003984365147630218)),4), (Array((mycket,0.008977479783453209), (bra,0.00818731006415827), (tjejer,0.007351349890882013), (kanske,0.007278806387254452), (tycker,0.005765657444255816), (tjej,0.005283260728111136), (aldrig,0.004850961525403287), (måste,0.004295311331401175), (många,0.004098909420309231), (fan,0.0037519527610768363)),5), (Array((kvinnor,0.010225491273158971), (män,0.009673480686118726), (tycker,0.007140266582380782), (mycket,0.006741668845739555), (många,0.006166286203067536), (tjejer,0.0055899906619138835), (kanske,0.005496960145787021), (bra,0.00450516316658287), (rätt,0.003975370248418236), (samma,0.003745046376784121)),6), (Array((tjej,0.007127353527224035), (tjejer,0.004957423825932039), (tycker,0.004956635485595906), (bra,0.004879988547583389), (aldrig,0.004864308362389227), (kanske,0.004588326800005052), (fan,0.0045304366858314), (mycket,0.004254791775847753), (kille,0.003227585234792242), (många,0.003104815800968856)),7), (Array((kanske,0.008605481820265399), (mycket,0.007793354439640682), (bra,0.007702749345394846), (tjejer,0.007368449734767341), (tycker,0.007331716366632319), (tjej,0.007143594626213114), (aldrig,0.0057960981929391165), (måste,0.004375258913740614), (killar,0.004267007182925261), (känner,0.004083723539468458)),8))

In [ ]:
// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

  

>     termRDD: org.apache.spark.rdd.RDD[(Array[(String, Double)], Int)] = ParallelCollectionRDD[15773] at parallelize at command-1716517738645101:2
>     termRDD2: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[15774] at flatMap at command-1716517738645101:3

In [ ]:
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  

>     termDF: org.apache.spark.sql.DataFrame = [term: string, probability: double ... 1 more field]

In [ ]:
display(termDF)

  

[TABLE]

Truncated to 30 rows

In [ ]:
// Create JSON data
val rawJson = termDF.toJSON.collect().mkString(",\n")

  

>     rawJson: String =
>     {"term":"com","probability":0.005828736389870819,"topicId":0},
>     {"term":"font","probability":0.005177770692548582,"topicId":0},
>     {"term":"kanske","probability":0.004281225684803414,"topicId":0},
>     {"term":"size","probability":0.003802661243134556,"topicId":0},
>     {"term":"mycket","probability":0.0037466618822440735,"topicId":0},
>     {"term":"tycker","probability":0.0035788236930912945,"topicId":0},
>     {"term":"tjej","probability":0.003572148658558219,"topicId":0},
>     {"term":"wmv","probability":0.003492052477056438,"topicId":0},
>     {"term":"fan","probability":0.003387032075470724,"topicId":0},
>     {"term":"killar","probability":0.0030759438164969536,"topicId":0},
>     {"term":"tjejer","probability":0.008472421218338572,"topicId":1},
>     {"term":"tycker","probability":0.00835186048039426,"topicId":1},
>     {"term":"mycket","probability":0.007028728954305847,"topicId":1},
>     {"term":"kvinnor","probability":0.006925949553647179,"topicId":1},
>     {"term":"bröst","probability":0.006778393528832462,"topicId":1},
>     {"term":"kanske","probability":0.006562072420550684,"topicId":1},
>     {"term":"bra","probability":0.005735098526572583,"topicId":1},
>     {"term":"tjej","probability":0.00509426741049249,"topicId":1},
>     {"term":"många","probability":0.0048473374351092275,"topicId":1},
>     {"term":"stora","probability":0.00476766751743259,"topicId":1},
>     {"term":"fan","probability":0.007136098623257581,"topicId":2},
>     {"term":"kanske","probability":0.006219005095591811,"topicId":2},
>     {"term":"bra","probability":0.00607101967985819,"topicId":2},
>     {"term":"bilder","probability":0.005716841064824449,"topicId":2},
>     {"term":"mycket","probability":0.00547781968634569,"topicId":2},
>     {"term":"tjej","probability":0.005384256034422819,"topicId":2},
>     {"term":"tycker","probability":0.005334349560442397,"topicId":2},
>     {"term":"tjejer","probability":0.004803912617599809,"topicId":2},
>     {"term":"riktigt","probability":0.004136561405063082,"topicId":2},
>     {"term":"måste","probability":0.004049538541763246,"topicId":2},
>     {"term":"the","probability":0.012209939662717557,"topicId":3},
>     {"term":"datanördar","probability":0.010448534134872305,"topicId":3},
>     {"term":"and","probability":0.00871415633044254,"topicId":3},
>     {"term":"you","probability":0.004717316738793793,"topicId":3},
>     {"term":"bra","probability":0.0036140424647016466,"topicId":3},
>     {"term":"kanske","probability":0.002794974212182175,"topicId":3},
>     {"term":"for","probability":0.00257613484350404,"topicId":3},
>     {"term":"mycket","probability":0.0025543316673021775,"topicId":3},
>     {"term":"tjejer","probability":0.0025314211222808034,"topicId":3},
>     {"term":"fan","probability":0.0021470545469380897,"topicId":3},
>     {"term":"kanske","probability":0.008136396956028923,"topicId":4},
>     {"term":"tycker","probability":0.007598480888602129,"topicId":4},
>     {"term":"barn","probability":0.0066038668672708575,"topicId":4},
>     {"term":"tjejer","probability":0.005629407585564553,"topicId":4},
>     {"term":"många","probability":0.004621223223798453,"topicId":4},
>     {"term":"aldrig","probability":0.004402198423276141,"topicId":4},
>     {"term":"bra","probability":0.004294088390266522,"topicId":4},
>     {"term":"mycket","probability":0.004118530511479333,"topicId":4},
>     {"term":"fan","probability":0.004054347059086492,"topicId":4},
>     {"term":"rätt","probability":0.003984365147630218,"topicId":4},
>     {"term":"mycket","probability":0.008977479783453209,"topicId":5},
>     {"term":"bra","probability":0.00818731006415827,"topicId":5},
>     {"term":"tjejer","probability":0.007351349890882013,"topicId":5},
>     {"term":"kanske","probability":0.007278806387254452,"topicId":5},
>     {"term":"tycker","probability":0.005765657444255816,"topicId":5},
>     {"term":"tjej","probability":0.005283260728111136,"topicId":5},
>     {"term":"aldrig","probability":0.004850961525403287,"topicId":5},
>     {"term":"måste","probability":0.004295311331401175,"topicId":5},
>     {"term":"många","probability":0.004098909420309231,"topicId":5},
>     {"term":"fan","probability":0.0037519527610768363,"topicId":5},
>     {"term":"kvinnor","probability":0.010225491273158971,"topicId":6},
>     {"term":"män","probability":0.009673480686118726,"topicId":6},
>     {"term":"tycker","probability":0.007140266582380782,"topicId":6},
>     {"term":"mycket","probability":0.006741668845739555,"topicId":6},
>     {"term":"många","probability":0.006166286203067536,"topicId":6},
>     {"term":"tjejer","probability":0.0055899906619138835,"topicId":6},
>     {"term":"kanske","probability":0.005496960145787021,"topicId":6},
>     {"term":"bra","probability":0.00450516316658287,"topicId":6},
>     {"term":"rätt","probability":0.003975370248418236,"topicId":6},
>     {"term":"samma","probability":0.003745046376784121,"topicId":6},
>     {"term":"tjej","probability":0.007127353527224035,"topicId":7},
>     {"term":"tjejer","probability":0.004957423825932039,"topicId":7},
>     {"term":"tycker","probability":0.004956635485595906,"topicId":7},
>     {"term":"bra","probability":0.004879988547583389,"topicId":7},
>     {"term":"aldrig","probability":0.004864308362389227,"topicId":7},
>     {"term":"kanske","probability":0.004588326800005052,"topicId":7},
>     {"term":"fan","probability":0.0045304366858314,"topicId":7},
>     {"term":"mycket","probability":0.004254791775847753,"topicId":7},
>     {"term":"kille","probability":0.003227585234792242,"topicId":7},
>     {"term":"många","probability":0.003104815800968856,"topicId":7},
>     {"term":"kanske","probability":0.008605481820265399,"topicId":8},
>     {"term":"mycket","probability":0.007793354439640682,"topicId":8},
>     {"term":"bra","probability":0.007702749345394846,"topicId":8},
>     {"term":"tjejer","probability":0.007368449734767341,"topicId":8},
>     {"term":"tycker","probability":0.007331716366632319,"topicId":8},
>     {"term":"tjej","probability":0.007143594626213114,"topicId":8},
>     {"term":"aldrig","probability":0.0057960981929391165,"topicId":8},
>     {"term":"måste","probability":0.004375258913740614,"topicId":8},
>     {"term":"killar","probability":0.004267007182925261,"topicId":8},
>     {"term":"känner","probability":0.004083723539468458,"topicId":8}

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")